# Objective
- This notebook is to transfer the codes developed by Tae to pytorch codes and some practical codes that can be used in real learning simulations.
- This code assumes real distributed environment.

Bottom Network or Server Network

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Concatenate,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.losses import mean_squared_error
import numpy as np
import math

class BottomNetwork(object):
    def __init__(self, config):
        self.task: str = config['task']
        self.bottom_input_size: int = config['bottom_input_size']
        self.layers: int = config['layers']
        self.hidden_units: int = config['hidden_units']
        self.random_seed: int = config['random_seed']
        self.bottom_network: Model = None # create_bottom_network
        self.bottom_weights_grads: tf.Tensor = None # bottom_split_gradients

    def create_bottom_network(self):
        input_layer = Input(shape=self.bottom_input_size, name="bottom_input")
        dense = input_layer
        for i in range(self.layers):
            dense = Dense(
                units=self.hidden_units,
                kernel_initializer=glorot_uniform(seed=self.random_seed),
                activation='relu',
                name='bottom_dense_{}'.format(i + 1)
            )(dense)
        self.bottom_network = Model(input_layer, dense, name=f"{self.task}_bottom_model")
        return self.bottom_network

    def bottom_split_gradients(self, bottom_input: np.ndarray, h_grad_from_top: tf.Tensor):
        with tf.GradientTape(persistent=True) as tape:
            bottom_input_tf = tf.constant(bottom_input)
            bottom_weights = self.bottom_network.trainable_weights
            tape.watch(bottom_input_tf)
            tape.watch(bottom_weights)
            h = self.bottom_network(bottom_input) # TODO h 두 번 계산됨. self에 넣고. bottom class 계산 --> Top Network에 전달?
        self.bottom_weights_grads = tape.gradient(h, bottom_weights, output_gradients=h_grad_from_top)
        return self.bottom_weights_grads

In [21]:
# %%writefile ../cloud2edge/utils/__init__.py


Writing ../cloud2edge/utils/__init__.py


In [24]:
%%writefile ../cloud2edge/configurer.py


class Configurer:
    def __init__(self, task, server_input_size, layers, hidden_units, random_seed):
        self.task = task:str
        self.server_input_size = server_input_size
        self.hidden_units = hidden_units
        self.random_seed = random_seed

Writing ../cloud2edge/configurer.py


In [25]:
%%writefile ../cloud2edge/models/server/server.py

import torch 
import torch.nn as nn
import torch.optim as optim
from collections import OrderedDict


class ServerNetwork:
    def __init__(self, config_object):
        self.task: str = config_object.task
        self.server_input_size = config_object.server_input_size
        self.layers = config_object.layers
        self.hidden_units = config_object.hidden_units
        self.random_seed = config_object.random_seed
        
        self.block_list = OrderedDict()
        
        self.server_network = None # create server network 
    
    def create_server_network(self):
        self.server_input_layer = nn.Linear(self.server_input_size, self.hidden_units)
        
        for idx, layer in enumerate(self.layers):
            
            self.block_list[str(idx)] = nn.Sequential(
                        nn.Linear(self.hidden_units, self.hidden_units),
                        nn.ReLU()
                        )
        return nn.Sequential(self.server_input_layer,
                            *self.block_list)
    
    def server_split_gradients(self, )


Overwriting ../cloud2edge/models/server/server.py
